In [1]:
from vllm import LLM
# model = "berkeley-nest/Starling-LM-7B-alpha"
model = "microsoft/Orca-2-13b"
llm = LLM(model = model, gpu_memory_utilization = 0.80)

config.json:   0%|          | 0.00/603 [00:00<?, ?B/s]

INFO 02-24 15:58:42 llm_engine.py:79] Initializing an LLM engine with config: model='microsoft/Orca-2-13b', tokenizer='microsoft/Orca-2-13b', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

...

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/9.87G [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/2.49G [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

INFO 02-24 16:07:15 llm_engine.py:337] # GPU blocks: 1067, # CPU blocks: 327
INFO 02-24 16:07:17 model_runner.py:676] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-24 16:07:17 model_runner.py:680] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-24 16:07:23 model_runner.py:748] Graph capturing finished in 6 secs.


In [61]:
# Orca 2.0 prompting strategies

from vllm.sampling_params import SamplingParams
from itertools import chain
context = '''In a nation, the law can serve to (1) keep the peace, (2) maintain the status quo, (3) preserve individual rights, (4) protect minorities against majorities, (5) promote social justice, and (6) provide for orderly social change. Some legal systems serve these purposes better than others. Although a nation ruled by an authoritarian government may keep the peace and maintain the status quo, it may also oppress minorities or political opponents (e.g., Burma, Zimbabwe, or Iraq under Saddam Hussein). Under colonialism, European nations often imposed peace in countries whose borders were somewhat arbitrarily created by those same European nations. Over several centuries prior to the twentieth century, empires were built by Spain, Portugal, Britain, Holland, France, Germany, Belgium, and Italy. With regard to the functions of the law, the empire may have kept the peace—largely with force—but it changed the status quo and seldom promoted the native peoples’ rights or social justice within the colonized nation.

In nations that were former colonies of European nations, various ethnic and tribal factions have frequently made it difficult for a single, united government to rule effectively. In Rwanda, for example, power struggles between Hutus and Tutsis resulted in genocide of the Tutsi minority. (Genocide is the deliberate and systematic killing or displacement of one group of people by another group. In 1948, the international community formally condemned the crime of genocide.) In nations of the former Soviet Union, the withdrawal of a central power created power vacuums that were exploited by ethnic leaders. When Yugoslavia broke up, the different ethnic groups—Croats, Bosnians, and Serbians—fought bitterly for home turf rather than share power. In Iraq and Afghanistan, the effective blending of different groups of families, tribes, sects, and ethnic groups into a national governing body that shares power remains to be seen.'''

text_info = '''Business Law and the Legal Environment (Saylor Academy, 2022) provides context and essential concepts across the entire range of legal issues with which managers and business executives must grapple. The text provides the vocabulary and legal acumen necessary for businesspeople to talk in an educated way to their customers, employees, suppliers, government officials—and to their own lawyers.'''

chat_history = [
    ("student", "Who are you?"),
    ("iTELL AI", "I am iTELL AI, a reading support agent that helps students with Business Law. I can answer your questions about the text or explain difficult concepts. I can also help you find other sources of information, such as online videos, podcasts, or articles. My goal is to assist you in learning Business Law.")
]

chat_history = {}

user_summary = "This text is about the different functions of the law in a nation. It says that the law can help to keep the peace, maintain the status quo, preserve individual rights, protect minorities, promote social justice, and provide for orderly social change. It also gives some examples of how different legal systems can affect the stability and diversity of a nation, such as the effects of colonialism, ethnic conflicts, and power vacuums."

def construct_prompt(user_message: str, text_name: str, text_info: str, context: str, chat_history:dict, user_summary: str):
    system_message = (
        f"You are iTELL AI, a reading support agent that helps students with an instructional text called {text_name}."
        f" {text_info}"
        "\niTELL stands for intelligent texts for enhanced lifelong learning. After reading a page in iTELL, the student must submit a summary of that page."
        " iTELL AI will try to help help students understand the text, but iTELL AI will not write any summaries for the student."
        " If the student asks iTELL AI for a summary, iTELL AI will tell the student that it cannot write the summary for them."
        " iTELL AI cannot provide any hyperlinks to external resources."
        " iTELL AI is factual and concise. If iTELL AI does not know the answer to a question, it truthfully says that it does not know."
    )

    if context:
        system_message += f"\nSTART CONTEXT BLOCK\n{context}\nEND OF CONTEXT BLOCK"

    if user_summary:
        system_message += f"\nSTART STUDENT SUMMARY\n{user_summary}\nEND STUDENT SUMMARY"

    sample_responses = [
        ("student", "What can you do for me?"),
        ("iTELL AI", f"I am an AI assistant which helps answer questions about {text_name}, but I cannot write summaries for you."),
        ("student", "Summarize the page."),
        ("iTELL AI", "Sorry, but I can't write any summaries for you. Please try asking another question."),
        ("student", "What do you think about politics?"),
        ("iTELL AI", f"Sorry, I don't like to talk about politics. Would you like to ask me a question about {text_name}?"),
    ]

    sample_dialogue = "\nSTART SAMPLE DIALOGUE"
    for agent, msg in chain(sample_responses, chat_history):
        sample_dialogue += f"\n{agent}\n{msg}"
    sample_dialogue += "\nEND SAMPLE DIALOGUE"

    # system_message += f"\nSTART SAMPLE DIALOGUE{sample_dialogue}\nEND SAMPLE DIALOGUE"

    prompt = (
        f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>system{sample_dialogue}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>iTELL AI"
    )

    return prompt

def generate(message):
    prompt = construct_prompt(message, "Business Law", text_info, context, chat_history, None)
    sampling_params = SamplingParams(stop=["<|im_end|>"], max_tokens=256)
    output = llm.generate(prompt, sampling_params)

    print(prompt)
    print("*"*80)
    print(output[0].outputs[0].text.strip())


generate("Please summarize the page.")

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.45s/it]

<|im_start|>system
You are iTELL AI, a reading support agent that helps students with an instructional text called Business Law. Business Law and the Legal Environment (Saylor Academy, 2022) provides context and essential concepts across the entire range of legal issues with which managers and business executives must grapple. The text provides the vocabulary and legal acumen necessary for businesspeople to talk in an educated way to their customers, employees, suppliers, government officials—and to their own lawyers.
iTELL stands for intelligent texts for enhanced lifelong learning. After reading a page in iTELL, the student must submit a summary of that page. iTELL AI will try to help help students understand the text, but iTELL AI will not write any summaries for the student. If the student asks iTELL AI for a summary, iTELL AI will tell the student that it cannot write the summary for them. iTELL AI cannot provide any hyperlinks to external resources. iTELL AI is factual and concis

In [66]:
text_name = "Business Law and the Legal Environment"
excerpt_chunk = context
question_type = "elaboration"
question_type_definition = "the process of making inferences that link what is in the text or sentence to related to a reader’s background knowledge. Readers use specific prior knowledge or learned experiences to understand a text by developing inferences based on specific background knowledge."

sampling_params = SamplingParams(stop=["<|im_end|>"], max_tokens=256)

llm.generate(
    "<|im_start|>system"
    "\nWrite a free response question based on the following highlighted chunk"
    f' from an instructional text titled "{text_name}".'
    " It is important to understand that free response questions are designed to elicit"
    " one of five cognitive processes from readers:"
    " Paraphrasing, Elaboration, Logic, Prediction, and Bridging."
    "\n[START HIGHLIGHTED CHUNK]"
    f"\n{excerpt_chunk}"
    "\n[END HIGHLIGHTED CHUNK]"
    f"\nNow, you will need to generate a {question_type} question based on the highlighted chunk."
    f" In this context, {question_type} means {question_type_definition}"
    " Write only one question and do not provide any opening, closing, or explanations.<|im_end|>"
    "\n<|im_start|>assistant",
    sampling_params
)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.72s/it]


[RequestOutput(request_id=59, prompt='<|im_start|>system\nWrite a free response question based on the following highlighted chunk from an instructional text titled "Business Law and the Legal Environment". It is important to understand that free response questions are designed to elicit one of five cognitive processes from readers: Paraphrasing, Elaboration, Logic, Prediction, and Bridging.\n[START HIGHLIGHTED CHUNK]\nIn a nation, the law can serve to (1) keep the peace, (2) maintain the status quo, (3) preserve individual rights, (4) protect minorities against majorities, (5) promote social justice, and (6) provide for orderly social change. Some legal systems serve these purposes better than others. Although a nation ruled by an authoritarian government may keep the peace and maintain the status quo, it may also oppress minorities or political opponents (e.g., Burma, Zimbabwe, or Iraq under Saddam Hussein). Under colonialism, European nations often imposed peace in countries whose bo

In [74]:
# Starling and OpenChat model prompting strategies.

from vllm.sampling_params import SamplingParams

chunk = '''In a nation, the law can serve to (1) keep the peace, (2) maintain the status quo, (3) preserve individual rights, (4) protect minorities against majorities, (5) promote social justice, and (6) provide for orderly social change. Some legal systems serve these purposes better than others. Although a nation ruled by an authoritarian government may keep the peace and maintain the status quo, it may also oppress minorities or political opponents (e.g., Burma, Zimbabwe, or Iraq under Saddam Hussein). Under colonialism, European nations often imposed peace in countries whose borders were somewhat arbitrarily created by those same European nations. Over several centuries prior to the twentieth century, empires were built by Spain, Portugal, Britain, Holland, France, Germany, Belgium, and Italy. With regard to the functions of the law, the empire may have kept the peace—largely with force—but it changed the status quo and seldom promoted the native peoples’ rights or social justice within the colonized nation.

In nations that were former colonies of European nations, various ethnic and tribal factions have frequently made it difficult for a single, united government to rule effectively. In Rwanda, for example, power struggles between Hutus and Tutsis resulted in genocide of the Tutsi minority. (Genocide is the deliberate and systematic killing or displacement of one group of people by another group. In 1948, the international community formally condemned the crime of genocide.) In nations of the former Soviet Union, the withdrawal of a central power created power vacuums that were exploited by ethnic leaders. When Yugoslavia broke up, the different ethnic groups—Croats, Bosnians, and Serbians—fought bitterly for home turf rather than share power. In Iraq and Afghanistan, the effective blending of different groups of families, tribes, sects, and ethnic groups into a national governing body that shares power remains to be seen.'''


def construct_prompt(message, text_name, text_info, chunk):
    preface = (
        f"GPT4 Correct Assistant is a reading support agent that helps students with an instructional text called {text_name}."
        " GPT4 is factual and concise. If GPT4 does not know the answer to a question, it truthfully says that it does not know."
        " GPT4 will not provide a summary of the context because GPT4 wants the User to read and understand the text on their own."
        "\nSTART CONTEXT BLOCK"
        f"\n{chunk}"
        "\nEND OF CONTEXT BLOCK\n"
    )

    rag = (
        f"GPT4 Correct User: The following is a highlighted section from {text_name}:\n{chunk}<|end_of_turn|>"

    )
    # preface = (
    #     f"GPT4 Correct User: I am currently reading an instructional text called {text_name}."
    #     " Do not provide a summary of the page for me.<|end_of_turn|>"
    #     "GPT4 Correct Assistant: Great, I will be happy to help.<|end_of_turn|>"
    # )
    sample_conversation = (
        'GPT4 Correct User: What can you do for me?<|end_of_turn|>'
        'GPT4 Correct Assistant: I am an AI assistant which helps answer questions'
        f' based on {text_name}.<|end_of_turn|>'
        'GPT4 Correct User: What do you think about politics?"<|end_of_turn|>'
        f'GPT4 Correct Assistant: Sorry, I don\'t like to talk about politics. Would you like to ask me a question about {text_name}?<|end_of_turn|>'
    )

    user_message = f'GPT4 Correct User: {message}<|end_of_turn|>GPT4 Correct Assistant: '

    return preface+user_message

def generate(message):
    prompt = construct_prompt(message, "Business Law", "The best book ever", chunk)
    sampling_params = SamplingParams(stop=["<|end_of_turn|>"], max_tokens=256)
    output = llm.generate(prompt, sampling_params)

    print(prompt)
    print("*"*80)
    print(output[0].outputs[0].text.strip())


generate("")

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.07s/it]

GPT4 Correct Assistant is a reading support agent that helps students with an instructional text called Business Law. GPT4 is factual and concise. If GPT4 does not know the answer to a question, it truthfully says that it does not know. GPT4 will not provide a summary of the context because GPT4 wants the User to read and understand the text on their own.
START CONTEXT BLOCK
In a nation, the law can serve to (1) keep the peace, (2) maintain the status quo, (3) preserve individual rights, (4) protect minorities against majorities, (5) promote social justice, and (6) provide for orderly social change. Some legal systems serve these purposes better than others. Although a nation ruled by an authoritarian government may keep the peace and maintain the status quo, it may also oppress minorities or political opponents (e.g., Burma, Zimbabwe, or Iraq under Saddam Hussein). Under colonialism, European nations often imposed peace in countries whose borders were somewhat arbitrarily created by t